# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_data=pd.read_csv("clean_city_data.csv")
clean_city_data

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,jumla,29,82,43,26,0,2,NP,1611717144
1,tessalit,20,1,63,25,0,10,ML,1611717023
2,isiro,2,27,64,96,68,2,CD,1611717144
3,rikitea,-23,-134,80,74,10,10,PF,1611716968
4,alekseyevsk,57,108,-26,82,99,3,RU,1611717144
...,...,...,...,...,...,...,...,...,...
559,jacareacanga,-6,-57,74,96,97,1,BR,1611717179
560,bambanglipuro,-7,110,80,79,100,7,ID,1611717179
561,beyneu,45,55,28,94,7,14,KZ,1611717179
562,necochea,-38,-58,64,82,100,11,AR,1611716976


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)

In [16]:
locations=clean_city_data[["Lat","Lng"]]
rating=clean_city_data["Humidity"].astype(float)

In [17]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
city_temp=clean_city_data.loc[(clean_city_data["Max_Temp"]<=80) & (clean_city_data["Max_Temp"]>=70),:]

city_temp.head()

# city_data_params.to_csv("city_data_params",index=False)

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
3,rikitea,-23,-134,80,74,10,10,PF,1611716968
5,mar del plata,-38,-57,71,83,75,4,AR,1611716964
6,zhongshan,21,110,70,65,0,11,CN,1611717144
13,mahebourg,-20,57,71,88,20,8,MU,1611716759
15,geraldton,-28,114,72,62,45,17,AU,1611716775


In [7]:
wind_speed=city_temp.loc[city_temp["Wind_Speed"]<=10]
wind_speed

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
3,rikitea,-23,-134,80,74,10,10,PF,1611716968
5,mar del plata,-38,-57,71,83,75,4,AR,1611716964
13,mahebourg,-20,57,71,88,20,8,MU,1611716759
26,farafangana,-22,47,75,91,42,8,MG,1611717145
40,cabo rojo,18,-67,75,94,2,7,PR,1611717146
...,...,...,...,...,...,...,...,...,...
545,candolim,15,73,74,60,9,5,IN,1611717178
548,trinidad,-14,-64,74,96,100,3,BO,1611717027
559,jacareacanga,-6,-57,74,96,97,1,BR,1611717179
560,bambanglipuro,-7,110,80,79,100,7,ID,1611717179


In [8]:
city_params=wind_speed.loc[wind_speed["Cloudiness"]==0]
city_params

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
77,imbituba,-28,-48,77,94,0,6,BR,1611717037
105,saint-philippe,-21,55,77,53,0,10,RE,1611716756
137,vila velha,-20,-40,77,83,0,4,BR,1611717052
161,puerto escondido,15,-97,77,70,0,4,MX,1611716981
207,saint-pierre,-21,55,77,53,0,10,RE,1611716778
271,salalah,17,54,73,31,0,10,OM,1611716862
305,sao filipe,14,-24,72,70,0,3,CV,1611716759
414,boa vista,2,-60,80,69,0,8,BR,1611717170
505,west bay,19,-81,79,78,0,5,KY,1611717175
515,panaba,21,-88,71,93,0,7,MX,1611717176


In [9]:
city_params.to_csv("city_data_params",index=False)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = city_params
hotel_df["Hotel Name"] = ""

C:\Users\cjbei\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [20]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "type": "lodging",
    "key": g_key,
}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
#     city = row['City']
    
    params["location"]= f"{lat},{lng}"
#     params["city"]=city
    
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


C:\Users\cjbei\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


In [21]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
77,imbituba,-28,-48,77,94,0,6,BR,1611717037,
105,saint-philippe,-21,55,77,53,0,10,RE,1611716756,
137,vila velha,-20,-40,77,83,0,4,BR,1611717052,
161,puerto escondido,15,-97,77,70,0,4,MX,1611716981,
207,saint-pierre,-21,55,77,53,0,10,RE,1611716778,
271,salalah,17,54,73,31,0,10,OM,1611716862,Hilton Salalah Resort
305,sao filipe,14,-24,72,70,0,3,CV,1611716759,
414,boa vista,2,-60,80,69,0,8,BR,1611717170,
505,west bay,19,-81,79,78,0,5,KY,1611717175,
515,panaba,21,-88,71,93,0,7,MX,1611717176,


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
# heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))